# Object GUI

## Choose the working directory

In [12]:
from ipyfilechooser import FileChooser
import ipywidgets as widgets
from IPython.display import clear_output
import os
import glob
import hickle
import importlib


# create initial widget
module_sel = widgets.Select()
module_sel.options = []

object_sel = widgets.Select()
object_sel.option = []
object_sel.rows=10

text_display = widgets.VBox([])

class ObjectGUI:
    def __init__(self, *args, **kwargs):
        if bool(glob.glob("config.hkl")):
            data = hickle.load("config.hkl")
            for (k, v) in data.items():
                setattr(self, k, v)
            # set up
            module_sel.options = self.module_list.keys()
            if module_sel.value == None:
                object_sel.option = []
            else:
                object_sel.options = self.module_list[module_sel.value]
                object_sel.value = self.module_list[module_sel.value][0]
            for module in module_sel.options:
                importlib.import_module(module)
            os.chdir(self.work_directory)
            
            # update argList
            module_import = importlib.import_module(module_sel.value)
            object_import = getattr(module_import, object_sel.value)
            argsList = getattr(object_import, 'argsList', 'None')
            text_list = []
            if argsList != 'None':
                for key in argsList:
                    if type(key[1]) == str:
                        a = widgets.Text(value = key[1],
                                     placeholder='Type something',
                                     description= key[0],
                                     disabled=False)
                        text_list += [a]
                    elif type(key[1]) == int:
                        a = widgets.IntText(
                                    value=key[1],
                                    description=key[0],
                                    disabled=False)
                        text_list += [a]
                    elif type(key[1]) == bool:
                        a = widgets.Checkbox(
                                    value=key[1],
                                    description=key[0],
                                    disabled=False)
                        text_list += [a]
                    else:
                        a = widgets.Text(value = str(key[1]),
                                     placeholder='Type something',
                                     description= key[0],
                                     disabled=False)
                        text_list += [a]
            text_display.children=tuple(text_list)
        else:
            self.module_list = {}
            self.work_directory = os.getcwd()
            self.config_directory = os.getcwd()
    
    def save(self):
        tosave = {}
        for (k, v) in self.__dict__.items():
            tosave[k] = v
        os.chdir(self.config_directory)
        hickle.dump(tosave, "config.hkl", mode="w")
        os.chdir(self.work_directory)


# create setting object        
config = ObjectGUI()
        
# Create new FileChooser:
# Path: current directory
# Show hidden files: no
fdialog = FileChooser(
    config.work_directory,
    filename='',
    title='<b>Change Directory</b>',
    show_hidden=False,
    select_default=True,
    use_dir_icons=True
)

display(fdialog)

cwd_out = widgets.Output()
display(cwd_out)
# Callback func
def change_directory(chooser):
    config.work_directory = chooser.selected
    os.chdir(chooser.selected)
    with cwd_out:
        display("Change directory to: " + os.getcwd())

# Register callback function
fdialog.register_callback(change_directory)


FileChooser(path='/Users/chris/Documents/GitHub/ObjectGUI', filename='', show_hidden='False')

Output()

## Choose Directory

In [13]:
fdialog2 = FileChooser(
    os.getcwd(),
    filename='',
    title='',
    change_desc = 'Choose',
    show_hidden=False,
    select_default=True,
    use_dir_icons=True
)

display(fdialog2)
display(module_sel)

FileChooser(path='/Users/chris/Documents/GitHub/ObjectGUI', filename='', show_hidden='False')

Select(options=(), value=None)

## Object

In [14]:
display(object_sel)

# Callback func
def search_module(chooser):
    cw = os.getcwd()
    os.chdir(chooser.selected)
    module_name = os.path.basename(os.path.normpath(chooser.selected))
    class_list = []
    # look for Module name folder
    if bool(glob.glob(module_name)):
        os.chdir(module_name)
        if bool(glob.glob("__init__.py")):
            ini_file = open("__init__.py")
            for line in ini_file.readlines():
                # skip empty lines
                line = line.strip()
                if not line:  # line is blank
                    continue
                last_word = line.split()[-1]
                if(last_word[0].isupper()):
                    class_list += [last_word]
            ini_file.close()
    # change possible selections
    config.module_list[module_name] = class_list
    module_sel.options = config.module_list.keys()
    module_sel.value = module_name
    display(module_sel)
    os.chdir(cw)

# Register callback function
fdialog2.register_callback(search_module)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        module_name = change['new']
        object_sel.options = config.module_list[module_name]
        object_sel.value = config.module_list[module_name][0]
        display(object_sel)

module_sel.observe(on_change)

Select(options=(), rows=10, value=None)

## Constructor Options

In [15]:
display(text_display)

module_import = None
object_import = None

def on_select_object(change):
    global module_import
    global object_import
    if change['type'] == 'change' and change['name'] == 'value':
        object_name = change['new']
        module_import = importlib.import_module(module_sel.value)
        object_import = getattr(module_import, object_name)
        argsList = getattr(object_import, 'argsList', 'None')
        text_list = []
        if argsList != 'None':
            for key in argsList:
                if type(key[1]) == str:
                    a = widgets.Text(value = key[1],
                                 placeholder='Type something',
                                 description= key[0],
                                 disabled=False)
                    text_list += [a]
                elif type(key[1]) == int:
                    a = widgets.IntText(
                                value=key[1],
                                description=key[0],
                                disabled=False)
                    text_list += [a]
                elif type(key[1]) == bool:
                    a = widgets.Checkbox(
                                value=key[1],
                                description=key[0],
                                disabled=False)
                    text_list += [a]
                else:
                    a = widgets.Text(value = str(key[1]),
                                 placeholder='Type something',
                                 description= key[0],
                                 disabled=False)
                    text_list += [a]
        text_display.children=tuple(text_list)

object_sel.observe(on_select_object)

updated_arg_list = {}
button = widgets.Button(description='Done')
out = widgets.Output()

def on_button_clicked(_):
    arg = text_display.children
    with out:
        global updated_arg_list
        updated_arg_list = {}
        for a in arg:
            updated_arg_list[a.description]=a.value
        display('Done')
            
button.on_click(on_button_clicked)            
display(button)
display(out)

VBox()

Button(description='Done', style=ButtonStyle())

Output()

## Object creation

In [16]:
proceed_button = widgets.Button(description='Proceed')
proceed_out = widgets.Output()

def on_proceed_button_clicked(_):
    with proceed_out:
        object_create = object_import(**updated_arg_list)
#         display(object_create.numSets)
        display('Done')
            
proceed_button.on_click(on_proceed_button_clicked)            
display(proceed_button)
display(proceed_out)

Button(description='Proceed', style=ButtonStyle())

Output()

In [ ]:
config_button = widgets.Button(description='Save configurations')
config_out = widgets.Output()

def on_config_button_clicked(_):
    with config_out:
        config.save()
        display('Saved')
            
config_button.on_click(on_config_button_clicked)            
display(config_button)
display(config_out)